# Cargar señales desde test.csv y realizar inferencia en hardware

Este notebook permite cargar señales desde un archivo CSV, visualizarlas y realizar inferencia utilizando el hardware conectado.

## 1. Importar librerías necesarias

Importamos pandas para manipulación de datos, matplotlib para visualización y las librerías necesarias para la comunicación con el hardware.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from time import sleep

# Importar la librería udma para comunicación con el hardware
import udma

## 2. Cargar señales desde test.csv

Leemos el archivo test.csv y convertimos cada fila en una lista de señales.

In [ ]:
# Ruta al archivo test.csv (ajustar si es necesario)
csv_path = "test.csv"

# Cargar el archivo CSV. Se asume que cada fila es una señal y cada columna un valor de muestra.
df = pd.read_csv(csv_path, header=None)

# Convertir cada fila en una lista de enteros (señal)
signals = df.values.tolist()

print(f"Se cargaron {len(signals)} señales desde {csv_path}")

## 3. Visualizar una señal de test

Seleccionamos una señal del conjunto de test y la graficamos para inspección visual.

In [ ]:
# Seleccionar el índice de la señal a visualizar
signal_idx = 0

signal = signals[signal_idx]

plt.figure(figsize=(13,2))
plt.plot(signal)
plt.title(f"Señal de test #{signal_idx}")
plt.xlabel("Muestras")
plt.ylabel("Amplitud (ADC units)")
plt.show()

## 4. Enviar señal de test al hardware y obtener inferencia

Utilizamos las funciones existentes para enviar la señal seleccionada al hardware y obtener el resultado de inferencia.

In [ ]:
# Parámetros de conexión (ajustar según corresponda)
IP_ADDRESS = '192.168.1.10'
IP_PORT = 7

# Inicializar la conexión con el hardware
pynq = udma.UDMA_CLASS(IP_ADDRESS, IP_PORT)

# Intentar conectar
connectionStatus = 0
while(connectionStatus == 0):
    connectionStatus = pynq.connect()
    sleep(1)

# Función para limpiar el FIFO de entrada
def clearInFifo(udmaInstance):
    udmaInstance.write_reg(17, 1)
    udmaInstance.write_reg(17, 0)

# Limpiar FIFO antes de enviar la señal
clearInFifo(pynq)

# Inicializar el IP core
pynq.write_reg(reg=1, data=1)

# Enviar la señal seleccionada
pynq.write_fifo(length=len(signal), Txdata=signal)[0]
pynq.write_reg(reg=2, data=1)
pynq.write_reg(reg=2, data=0)

# Leer el resultado de inferencia
CB_INFERENCE_REG = 2
inference_result = pynq.read_reg(reg=CB_INFERENCE_REG)[1][0]

print(f"Resultado de inferencia para la señal #{signal_idx}: {inference_result}")

## 5. Evaluar múltiples señales de test

Iteramos sobre varias señales del archivo test.csv, enviamos cada una al hardware y recolectamos los resultados de inferencia.

In [ ]:
# Número de señales a evaluar (puede ajustarse)
num_signals_to_test = 10

results = []

for idx in range(num_signals_to_test):
    test_signal = signals[idx]
    clearInFifo(pynq)
    pynq.write_reg(reg=1, data=1)
    pynq.write_fifo(length=len(test_signal), Txdata=test_signal)[0]
    pynq.write_reg(reg=2, data=1)
    pynq.write_reg(reg=2, data=0)
    result = pynq.read_reg(reg=CB_INFERENCE_REG)[1][0]
    results.append(result)
    print(f"Señal #{idx}: Inferencia = {result}")

# Los resultados quedan almacenados en la lista 'results'